__할 일__
  - 한글 토크나이저 적용
  - 패인된 시퀀스 텐서로 변환

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

완료!


In [66]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.6.0


In [7]:
!head  /aiffel/aiffel/seq2seq_gd/korean-english-park.train.en
!head  /aiffel/aiffel/seq2seq_gd/korean-english-park.train.ko

Much of personal computing is about "can you top this?"
so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.
Like all optical mice, But it also doesn't need a desk.
uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.
Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack.
After learning of several foiled al Qaeda attempts on U.S. and commercial ships in the area, experts are warning that the terror network still wants to cripple the global economy, the world's economic jugular vein in Southeast Asia is at risk.
Caffeine can help increase reaction time and improve performance for military servicemen who must perform complex tasks or who need help staying alert for longer periods of time, accordin

In [14]:
kor_file = "/aiffel/aiffel/seq2seq_gd/korean-english-park.train.ko"
eng_file = "/aiffel/aiffel/seq2seq_gd/korean-english-park.train.en"

with open(kor_file, "r") as f:
    kor_raw = f.read().splitlines()

with open(eng_file, "r") as f:
    eng_raw = f.read().splitlines()

print("Data Size:", len(kor_raw), len(eng_raw))
print("Example:")

Data Size: 94123 94123
Example:


In [15]:
for sen in kor_raw[0:100][::20]: print(">>", sen)

>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.


In [18]:
# 중복된 데이터를 제거
kor_eng_pairs = set(kor_eng for kor_eng in zip(kor_raw, eng_raw))
print("kor_eng_pairs: ", len(kor_eng_pairs))

kor_eng_pairs:  78968


In [23]:
def preprocess_sentence_kor(sentence, s_token=False, e_token=False):
    # (유니코드 문자열 ".", "?", "!", ",")를 제외한 모든 문자를 공백으로 대체
    # 한글은 유니코드에 포함되어 있고 낱소리만 쓰인 문자열도 포함됨
    sentence = re.sub(r'([^\w?!,])', " ", sentence)
    sentence = sentence.strip()

    # 구두점(punctuation)을 토큰으로 인식하도록 단어와의 사이에 공백
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r'[" "]+', " ", sentence)

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [24]:
def preprocess_sentence_eng(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [27]:
enc_corpus = []
dec_corpus = []

# num_examples = 30000

for pair in kor_eng_pairs:
    kor, eng = pair

    enc_corpus.append(preprocess_sentence_kor(kor))
    dec_corpus.append(preprocess_sentence_eng(eng, s_token=True, e_token=True))

print("Korean:", enc_corpus[100])   # go away !
print("English:", dec_corpus[100])   # <start> salga de aqu ! <end>

Korean: Report on the two Koreas relation in 20052005년 남북한 관계 총결산2006 02 원문 독해설명 인쇄
English: <start> the south korean government has consistently pursued progress in inter korean relations in harmony with the resolution of the north korean nuclear issue . <end>


In [31]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
# 토큰화하기; 한글 토크나이저 적용 필요
X, enc_tokenizer = tokenize(enc_corpus)

In [32]:
y, dec_tokenizer = tokenize(dec_corpus)

In [35]:
print(X[:5])

[[2396, 39705, 47, 6608, 53363, 5075, 16091, 4308, 53364, 3751, 13527, 53365, 739, 6], [53366, 2869, 14655, 3651], [6927, 1387, 2213, 6928, 31842, 1387, 3297, 20073, 1387, 4579, 39706, 1676, 20074, 1387, 274, 12492, 2], [3006, 2587, 2000, 26561, 8035, 494, 1677, 170, 31843, 2350, 53367, 2434, 9546, 26562, 5453, 13528, 24, 53368, 11, 2301, 551, 65], [39707, 83340, 220, 7619, 53369, 17864, 31844, 379, 75, 14656, 17865, 1526]]


In [38]:
X_over_cnt = 0
y_over_cnt = 0
over_cnt = 0

for Xy in zip(X, y):
    if len(Xy[0])>40: X_over_cnt+=1
    if len(Xy[1])>40: y_over_cnt+=1        
    if len(Xy[0])>40 and len(Xy[1])>40: over_cnt+=1        

In [39]:
print(X_over_cnt)
print(y_over_cnt)
print(over_cnt)

224
8727
161


In [73]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen=40

enc_cleaned = tf.compat.v1.placeholder(tf.float32, (None, maxlen))
dec_cleaned = tf.compat.v1.placeholder(tf.float32, (None, maxlen))
for Xy in zip(X, y):
  if len(Xy[1])>40:
    tf.concat(enc_cleaned, pad_sequences((Xy[0],), padding='pre', maxlen=maxlen))
    tf.concat(dec_cleaned, pad_sequences((Xy[1],), padding='pre', maxlen=maxlen))
    
#     enc_cleaned.extend(pad_sequences((Xy[0],), padding='pre', maxlen=40))
#     dec_cleaned.extend(pad_sequences((Xy[1],), padding='pre', maxlen=40))

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [46]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(
                                enc_cleaned, dec_cleaned, test_size=0.2, random_state=42)

In [48]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [50]:
class Encoder(tf.keras.Model): # 예제코드 재사용
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [64]:
class Decoder(tf.keras.Model): # 예제코드 재사용
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [53]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units         = 512 # 1024 절반
embedding_dim = 256 # 512 절반

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (64, 30, 512)
Decoder Output: (64, 44566)
Decoder Hidden State: (64, 512)
Attention: (64, 30, 1)


In [63]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [56]:
@tf.function  # 예제코드 재사용
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss


In [57]:
@tf.function  # 예제코드 재사용
def eval_step(src, tgt, encoder, decoder, dec_tok):
    bsz = src.shape[0]
    loss = 0

    enc_out = encoder(src)

    h_dec = enc_out[:, -1]
    
    dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

    for t in range(1, tgt.shape[1]):
        pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

        loss += loss_function(tgt[:, t], pred)
        dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))
    
    return batch_loss

In [61]:
from tqdm import tqdm
import random

EPOCHS = 3

for epoch in range(EPOCHS):  # 예제코드 재사용
    total_loss = 0
    
    idx_list = list(range(0, len(enc_train), BATCH_SIZE)) # enc_train.shape[0]
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                dec_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    test_loss = 0
    
    idx_list = list(range(0, len(enc_val), BATCH_SIZE)) # enc_val.shape[0]
    random.shuffle(idx_list)
    t = tqdm(idx_list)
    
    for (test_batch, idx) in enumerate(t):
        test_batch_loss = eval_step(enc_val[idx:idx+BATCH_SIZE],
                                    dec_val[idx:idx+BATCH_SIZE],
                                    encoder,
                                    decoder,
                                    dec_tokenizer)
    
        test_loss += test_batch_loss

        t.set_description_str('Test Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Test Loss %.4f' % (test_loss.numpy() / (test_batch + 1)))

  0%|          | 0/110 [00:00<?, ?it/s]


AttributeError: in user code:

    /tmp/ipykernel_33/2865613408.py:3 train_step  *
        bsz = src.shape[0]

    AttributeError: 'list' object has no attribute 'shape'


In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))


translate("Can I have some coffee?", encoder, decoder)